In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip -q install pyhrv

In [ ]:
folder_input_path = '/content/drive/My Drive/Colab Notebooks/Ch4_HR_HRV_Generation/data-HRV-physionet/'
output_output_path = '/content/drive/My Drive/Colab Notebooks/Ch4_HR_HRV_Generation/data-HRV-physionet/'
name = '013'

In [ ]:
import numpy as np
import pandas as pd
# from pyhrv import time_domain as td

def calculate_hrv_metrics(rr_intervals):
    mean_rr = np.mean(rr_intervals)
    median_rr = np.median(rr_intervals)
    sdrr = np.std(rr_intervals)
    differences = np.diff(rr_intervals)
    rmssd = np.sqrt(np.mean(differences**2)) # Root mean square of successive differences
    sdsd = np.std(differences) # Standard deviation of NN intervals
    sdrr_rmssd = sdrr / rmssd if rmssd != 0 else 0
    hr = 60000 / mean_rr if mean_rr != 0 else 0  # HR in beats per minute
    return mean_rr, median_rr, sdrr, rmssd, sdsd, sdrr_rmssd, hr

# def calculate_hrv_metrics_pyhrv(rr_intervals):
#   rmssd = td.rmssd(rr_intervals)  # Root mean square of successive differences
#   sdrr = td.sdnn(rr_intervals)  # Standard deviation of NN intervals

#   return sdrr, rmssd

def read_rr_intervals(file_path):
    # Read RR interval data from the file
    with open(file_path, 'r') as file:
      rr_intervals = []
      for line in file:
          stripped_line = line.strip()
          if stripped_line.isdigit():  # Check if the line contains digits only
              rr_intervals.append(int(stripped_line))

    return rr_intervals

def process_rr_intervals(rr_intervals):
    # Splitting the data into one-minute intervals
    minute_intervals = []
    current_interval = []
    time_sum = 0
    for rr in rr_intervals:
        time_sum += rr
        current_interval.append(rr)
        if time_sum >= 60000: # Sum up the milliseconds of individual RR records to get equal or over 60 s
            minute_intervals.append(current_interval)
            current_interval = []
            time_sum = 0
    if current_interval:
        minute_intervals.append(current_interval)

    # Calculating HRV metrics and preparing original RR interval data for CSV output
    hrv_metrics = []
    rr_interval_data = []
    for i, interval in enumerate(minute_intervals, 1):
        metrics = calculate_hrv_metrics(np.array(interval))
        hrv_metrics.append([i] + list(metrics))
        for rr in interval:
            rr_interval_data.append([i, rr])


    # Convert HRV metrics to DataFrame for CSV output
    hrv_columns = ["Interval_ID", "MEAN_RR", "MEDIAN_RR", "SDRR", "RMSSD", "SDSD", "SDRR_RMSSD", "HR"]
    hrv_df = pd.DataFrame(hrv_metrics, columns=hrv_columns)

    # Convert original RR interval data to DataFrame for CSV output
    rr_columns = ["Interval_ID", "RR_Interval"]
    rr_df = pd.DataFrame(rr_interval_data, columns=rr_columns)

    return hrv_df, rr_df

In [ ]:
# File path of the RR interval data
file_path = folder_input_path + name + '.txt'

rr_intervals = read_rr_intervals(file_path)

# Process the RR intervals and calculate HRV metrics
hrv_df, rr_df = process_rr_intervals(rr_intervals)

# Save the HRV metrics and original RR intervals to CSV files
hrv_csv_path = output_output_path + name + '_HRV.csv'
rr_csv_path = output_output_path + name + '_RR.csv'
hrv_df.to_csv(hrv_csv_path, index=False)
rr_df.to_csv(rr_csv_path, index=False)

print(f"HRV metrics saved to {hrv_csv_path}")
print(f"Original RR intervals saved to {rr_csv_path}")

HRV metrics saved to /content/drive/My Drive/Colab Notebooks/Ch4_HR_HRV_Generation/data-HRV-physionet/013_HRV.csv
Original RR intervals saved to /content/drive/My Drive/Colab Notebooks/Ch4_HR_HRV_Generation/data-HRV-physionet/013_RR.csv


In [ ]:
hrv_df.head()

,Interval_ID,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR
0,1,669.277778,679.0,31.299637,12.246990,12.244263,2.555700,89.648875
1,2,651.043011,649.0,18.057205,11.580062,11.580056,1.559336,92.159810
2,3,675.730337,680.0,33.892018,12.549900,12.511792,2.700581,88.792817
3,4,667.022222,664.0,28.993477,16.929464,16.906764,1.712605,89.952026
4,5,630.208333,633.0,19.678625,13.852988,13.851832,1.420533,95.206612


In [ ]:
rr_df.head(100)

,Interval_ID,RR_Interval
0,1,664
1,1,665
2,1,656
3,1,672
4,1,672
...,...,...
95,2,648
96,2,649
97,2,656
98,2,656


In [ ]:
# calculate_hrv_metrics_pyhrv(rr_intervals)